In [1]:
import numpy as np
import warnings


def swapRows(A, i, j):
    """
    interchange two rows of A
    operates on A in place
    """
    tmp = A[i].copy()
    A[i] = A[j]
    A[j] = tmp


def relError(a, b):
    """
    compute the relative error of a and b
    """
    with warnings.catch_warnings():
        warnings.simplefilter("error")
        try:
            return np.abs(a - b) / np.max(np.abs(np.array([a, b])))
        except:
            return 0.0


def rowReduce(A, i, j, pivot):
    """
    reduce row j using row i with pivot pivot, in matrix A
    operates on A in place
    """
    factor = A[j][pivot] / A[i][pivot]
    for k in range(len(A[j])):
        # we allow an accumulation of error 100 times larger 
        # than a single computation
        # this is crude but works for computations without a large 
        # dynamic range
        if relError(A[j][k], factor * A[i][k]) < 100 * np.finfo('float').resolution:
            A[j][k] = 0.0
        else:
            A[j][k] = A[j][k] - factor * A[i][k]


# stage 1 (forward elimination)
def forwardElimination(B):
    """
    Return the row echelon form of B
    """
    A = B.copy()
    m, n = np.shape(A)
    for i in range(m - 1):
        # Let lefmostNonZeroCol be the position of the leftmost nonzero value 
        # in row i or any row below it 
        leftmostNonZeroRow = m
        leftmostNonZeroCol = n
        ## for each row below row i (including row i)
        for h in range(i, m):
            ## search, starting from the left, for the first nonzero
            for k in range(i, n):
                if (A[h][k] != 0.0) and (k < leftmostNonZeroCol):
                    leftmostNonZeroRow = h
                    leftmostNonZeroCol = k
                    break
        # if there is no such position, stop
        if leftmostNonZeroRow == m:
            break
        # If the leftmostNonZeroCol in row i is zero, swap this row 
        # with a row below it
        # to make that position nonzero. This creates a pivot in that position.
        if (leftmostNonZeroRow > i):
            swapRows(A, leftmostNonZeroRow, i)
        # Use row reduction operations to create zeros in all positions 
        # below the pivot.
        for h in range(i + 1, m):
            rowReduce(A, i, h, leftmostNonZeroCol)
    return A


####################

# If any operation creates a row that is all zeros except the last element,
# the system is inconsistent; stop.
def inconsistentSystem(A):
    """
    B is assumed to be in echelon form; return True if it represents
    an inconsistent system, and False otherwise
    """
    rows = len(A)
    for k in range(0, rows):
        collumns = len(A[k])
        count = 0
        for j in range(0, collumns - 1):
            count += A[k][j]
        if (count == 0 and A[k][collumns - 1] != 0):
            return True;
    
    return False;
    


def backsubstitution(B):
    """
    return the reduced row echelon form matrix of B
    """
    A = B.astype(float)
    rows = len(A)
    col = len(A[0])
    # Devision algorithm to get 
    for k in range(0, rows):
        non_zero = 0
        for j in range(0, col-1):
            if A[k][j] != 0:
                non_zero = A[k][j]
                break
        for j in range(0, col):
            if A[k][j] != 0:
                A[k][j] = A[k][j] / non_zero
                
    for k in range(0, rows):
        non_zero = 0
        for j in range(0, col-1):
            if A[k][j] != 0:
                non_zero = j
                break
        for j in range(0, k):
            value =  - A[j][non_zero]
            for l in range(0, col):
                if A[j][l] != 0:
                    A[j][l] += A[k][l] * value
        for j in range(k+1, rows):
            value =  - A[j][non_zero]
            for l in range(0, col):
                if A[j][l] != 0:
                    A[j][l] += A[k][l] * value
    
    
    return A
    

#####################


A = np.array([[27.6, 30.2, 162], [3100, 6400, 23610], [250, 360, 1623]])
AEchelon = forwardElimination(A)
if (not inconsistentSystem(AEchelon)):
    AReducedEchelon = backsubstitution(AEchelon)


print(AReducedEchelon)

[[  8  11  -6  -7  13]
 [  0   1   0   0   2]
 [  0   0   1   0  -7]
 [  0   0   0   5 -12]]


In [4]:
import numpy as np

def innerProduct(a,b):
    value = 0
    for i in range(0, len(a)):
        value += a[i]*b[i]
        
    return value

def AxIP(A,x):
    b = [0 for j in range(0, len(A))]
    for i in range(0, len(A)):
        b[i] = innerProduct(A[i,:], x)
        
    return np.array(b)

def AxVS(A,x):
    b = [0 for j in range(0, len(A))]
    for i in range(0, len(A[0])):
        c = A[:,i]
        for k in range(0, len(A)):
            b[k] += c[k] * x[i]
        
    return np.array(b)

A1 = np.loadtxt('h4A1.txt')
x1 = np.loadtxt('h4x1.txt')
A2 = np.loadtxt('h4A2.txt')
x2 = np.loadtxt('h4x2.txt')


print(AxIP(A1,x1))
print(AxVS(A1,x1))
print()
print(AxIP(A2,x2))
print(AxVS(A2,x2))

[ 1.70000000e+01  1.30000000e+01  1.50000000e+01 -3.55271368e-15
  2.00000000e+00  4.00000000e+00]
[ 1.70000000e+01  1.30000000e+01  1.50000000e+01 -3.55271368e-15
  2.00000000e+00  4.00000000e+00]

[10.  9.  8.  7.  6.  5.]
[10.  9.  8.  7.  6.  5.]


In [2]:
def getPopulation(val):
    lst = [0,0]
    
    lst[0] = 0.94 * val[0] + 0.04 * val[1]
    lst[1] = 0.06 * val[0] + 0.96 * val[1]
    
    return lst

val2015 = [10000000, 800000]

val2016 = getPopulation(val2015)

val2017 = getPopulation(val2016)

print(val2015, val2016, val2017)
    
    
    

[10000000, 800000] [9432000.0, 1368000.0] [8920800.0, 1879200.0]


In [7]:
import math

def getCars(val):
    lst = [0,0,0]
    
    lst[0] = round(0.97 * val[0]) + round(0.05 * val[1]) + round(0.10 * val[2])
    lst[1] = round(0.00 * val[0]) + math.floor(0.90 * val[1]) + round(0.05 * val[2])
    lst[2] = round(0.03 * val[0]) + round(0.05 * val[1]) + round(0.85 * val[2])
    
    return lst

valM = [295, 55, 150]

valT = getCars(valM)

valW = getCars(valT)

print(valM, valT, valW)

[295, 55, 150] [304, 57, 140] [312, 58, 131]


In [1]:
import numpy as np

a = np.zeros((5,6))

b = np.ones((3,5))

c = np.identity(6)

d = np.zeros((5, 5))
np.fill_diagonal(d, [3,5,7,2,4])

print(a)
print()
print(b)
print()
print(c)
print()
print(d)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]

[[3. 0. 0. 0. 0.]
 [0. 5. 0. 0. 0.]
 [0. 0. 7. 0. 0.]
 [0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 4.]]



In [2]:
a = np.random.rand(6,4)
        
b = np.random.randint(-9, 10, size=(3,3))

print(a)
print()
print(b)

[[0.85686825 0.09540811 0.63970254 0.08333437]
 [0.6164653  0.23919012 0.12196163 0.76965196]
 [0.30556139 0.69166998 0.04790988 0.35300464]
 [0.41213625 0.99896008 0.32030461 0.0401705 ]
 [0.08930268 0.98105096 0.51016964 0.17446602]
 [0.04921934 0.05253567 0.43313012 0.88371839]]

[[-6  1  9]
 [ 4  9 -5]
 [ 3  5 -7]]


In [2]:
import numpy as np

def computeVal(A,B):
    C = np.add(A,B)
    D = np.subtract(A,B)
    
    E = np.matmul(C, D)
    
    F = np.matmul(A,A)
    G = np.subtract(F,B)
    
    return np.subtract(E,G)

def computeVal2A(A,B):
    C = np.add(A,B)
    D = np.subtract(A,B)
    E = np.matmul(C, D)
    
    F = np.matmul(A,A)
    G = np.matmul(B,B)
    H = np.subtract(F,G)
    
    return np.subtract(E,H)

def compareVal(A,B):
    C = computeVal2A(A,B)
    D = np.zeros((4,4))
    return np.array_equal(C,D)

A = np.random.randint(0, 100, size=(4,4))
B = np.random.randint(0, 100, size=(4,4))
C = np.random.randint(0, 100, size=(4,4))
I = np.identity(4)

g = computeVal(A,I)
h = computeVal(B,I)
i = computeVal(C,I)

D1 = np.random.randint(0, 100, size=(4,4))
D2 = np.random.randint(0, 100, size=(4,4))
E1 = np.random.randint(0, 100, size=(4,4))
E2 = np.random.randint(0, 100, size=(4,4))
F1 = np.random.randint(0, 100, size=(4,4))
F2 = np.random.randint(0, 100, size=(4,4))

j = compareVal(D1, D2)
k = compareVal(E1, E2)
l = compareVal(F1, F2)

print(g)
print()
print(h)
print()
print(i)
print()
print(j, k, l)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

False False False


In [20]:
import numpy as np
import warnings

def swapRows(A, i, j):
    """
    interchange two rows of A
    operates on A in place
    """
    tmp = A[i].copy()
    A[i] = A[j]
    A[j] = tmp

def relError(a, b):
    """
    compute the relative error of a and b
    """
    with warnings.catch_warnings():
        warnings.simplefilter("error")
        try:
            return np.abs(a-b)/np.max(np.abs(np.array([a, b])))
        except:
            return 0.0

def rowReduce(A, i, j, pivot):
    """
    reduce row j using row i with pivot pivot, in matrix A
    operates on A in place
    """
    factor = A[j][pivot] / A[i][pivot]
    for k in range(len(A[j])):
        # we allow an accumulation of error 100 times larger than a single computation
        # this is crude but works for computations without a large dynamic range
        if relError(A[j][k], factor * A[i][k]) < 100 * np.finfo('float').resolution:
            A[j][k] = 0.0
        else:
            A[j][k] = A[j][k] - factor * A[i][k]

# stage 1 (forward elimination)
def forwardElimination(B):
    """
    Return the row echelon form of B
    """
    A = B.copy().astype(float)
    m, n = np.shape(A)
    for i in range(m-1):
        # Let lefmostNonZeroCol be the position of the leftmost nonzero value 
        # in row i or any row below it 
        leftmostNonZeroRow = m
        leftmostNonZeroCol = n
        ## for each row below row i (including row i)
        for h in range(i,m):
            ## search, starting from the left, for the first nonzero
            for k in range(i,n):
                if (A[h][k] != 0.0) and (k < leftmostNonZeroCol):
                    leftmostNonZeroRow = h
                    leftmostNonZeroCol = k
                    break
        # if there is no such position, stop
        if leftmostNonZeroRow == m:
            break
        # If the leftmostNonZeroCol in row i is zero, swap this row 
        # with a row below it
        # to make that position nonzero. This creates a pivot in that position.
        if (leftmostNonZeroRow > i):
            swapRows(A, leftmostNonZeroRow, i)
        # Use row reduction operations to create zeros in all positions 
        # below the pivot.
        for h in range(i+1,m):
            rowReduce(A, i, h, leftmostNonZeroCol)
    return A

#################### 

# If any operation creates a row that is all zeros except the last element,
# the system is inconsistent; stop.
def inconsistentSystem(A):
    """
    B is assumed to be in echelon form; return True if it represents
    an inconsistent system, and False otherwise
    """
    m, n = np.shape(A)
    for i in range(m):
        for j in range(n):
            if (A[i][j] != 0):
                if (j == n-1):
                    return True
                else:
                    break
    return False

def backsubstitution(B):
    """
    return the reduced row echelon form matrix of B
    """
    A = B.copy().astype(float)
    m, n = np.shape(A)
    for i in range(m):
        # If row i is all zeros, or if i exceeds the number of rows in A, stop.
        for j in range(n):
            if (A[i][j] != 0.0):
                break
        if (j == n-1):
            return A
        pivot = j
        # If row i has a nonzero pivot value, divide row i by its pivot value.
        # This creates a 1 in the pivot position.
        A[i] = A[i] / A[i][pivot]
        for j in range(i):
            rowReduce(A, i, j, pivot)
    return A


#####################



A = np.array([[-25, -9, -27], [546,  180, 537], [154, 50, 149]])
print("Let A = \n", A, "\n") 
B = np.array([[0,0], [1,0],[0,1]])
print("Let B = \n", B, "\n") 
C = np.array([[-25, -9, -27, 0, 0], [546,  180, 537, 1, 0], [154, 50, 149, 0, 1]])
print("Let C = [A B] = \n", C, "\n") 

CEchelon = forwardElimination(C)
if (not inconsistentSystem(CEchelon)):
    CReducedEchelon = backsubstitution(CEchelon)

print("If we put C in reduced row echelon we get that C = \n", CReducedEchelon, "\n")
print("So the final two columns of the inverse of A = \n", CReducedEchelon[:,3:], "\n")

Let A = 
 [[-25  -9 -27]
 [546 180 537]
 [154  50 149]] 

Let B = 
 [[0 0]
 [1 0]
 [0 1]] 

Let C = [A B] = 
 [[-25  -9 -27   0   0]
 [546 180 537   1   0]
 [154  50 149   0   1]] 

If we put C in reduced row echelon we get that C = 
 [[  1.           0.           0.           1.5         -4.5       ]
 [  0.           1.           0.         -72.16666667 219.5       ]
 [ -0.          -0.           1.          22.66666667 -69.        ]] 

So the final two columns of the inverse of A = 
 [[  1.5         -4.5       ]
 [-72.16666667 219.5       ]
 [ 22.66666667 -69.        ]] 

